In [1]:

import nltk
from nltk.stem.isri import ISRIStemmer

# النص الخام
text = "يا جماعة إحنا بدنا نشتغل صح عشان المشروع ينجح الزلمة اللي كان هنا مبارح زعلان مرة إيش نسوي هسا لازم نشوف حل الطلاب يدرسون بجدية"

# 1. المعالج التقليدي
stemmer_old = ISRIStemmer()

print(f"{'الكلمة':<10} | {'الجذر (ISRI التقليدي)':<20}")
print("-" * 35)

for word in text.split():
    # محاولة التجذير مباشرة
    root = stemmer_old.stem(word)
    print(f"{word:<10} | {root:<20}")


الكلمة     | الجذر (ISRI التقليدي)
-----------------------------------
يا         | يا                  
جماعة      | جمع                 
إحنا       | احن                 
بدنا       | بدن                 
نشتغل      | شتغل                
صح         | صح                  
عشان       | عشن                 
المشروع    | شرع                 
ينجح       | نجح                 
الزلمة     | زلم                 
اللي       | الل                 
كان        | كان                 
هنا        | هنا                 
مبارح      | برح                 
زعلان      | زعل                 
مرة        | مرة                 
إيش        | ايش                 
نسوي       | نسي                 
هسا        | هسا                 
لازم       | لزم                 
نشوف       | نشف                 
حل         | حل                  
الطلاب     | طلب                 
يدرسون     | درس                 
بجدية      | بجد                 


In [2]:

import re
from nltk.stem.isri import ISRIStemmer

class SmartISRI(ISRIStemmer):
    def __init__(self):
        super().__init__()
        # قاموس العقل المدبر (يحل مشاكل العامية)
        self.dialect_map = {
            "إحنا": "نحن",
            "بدنا": "نريد",      
            "عشان": "لأجل",
            "الزلمة": "الرجل",
            "مبارح": "أمس",
            "مرة": "جدا",       
            "إيش": "ماذا",
            "نسوي": "نفعل",     
            "هسا": "الآن",
            "نشوف": "نرى"
        }

    def smart_stem(self, word):
        # 1. التنظيف المبدئي
        word_clean = re.sub("[إأآ]", "ا", word) # توحيد الألفات
        
        # 2. البحث في قاموس اللهجات
        # إذا وجد الكلمة في القاموس، يستبدلها بالفصحى، وإلا يبقيها كما هي
        msa_word = self.dialect_map.get(word_clean, word_clean)
        
        # 3. استخدام خوارزمية ISRI القوية على الكلمة الفصحى
        root = super().stem(msa_word)
        
        return msa_word, root
 

In [3]:

# --- التنفيذ ---
smart_stemmer = SmartISRI()

print("\n" + "="*60)
print(f"{'الكلمة الأصلية':<15} | {'المعنى الفصحى':<15} | {'الجذر الذكي':<10}")
print("="*60)

for word in text.split():
    msa, root = smart_stemmer.smart_stem(word)
    print(f"{word:<15} | {msa:<15} | {root:<10}")


الكلمة الأصلية  | المعنى الفصحى   | الجذر الذكي
يا              | يا              | يا        
جماعة           | جماعة           | جمع       
إحنا            | احنا            | احن       
بدنا            | نريد            | نرد       
نشتغل           | نشتغل           | شتغل      
صح              | صح              | صح        
عشان            | لأجل            | أجل       
المشروع         | المشروع         | شرع       
ينجح            | ينجح            | نجح       
الزلمة          | الرجل           | رجل       
اللي            | اللي            | الل       
كان             | كان             | كان       
هنا             | هنا             | هنا       
مبارح           | أمس             | امس       
زعلان           | زعلان           | زعل       
مرة             | جدا             | جدا       
إيش             | ايش             | ايش       
نسوي            | نفعل            | فعل       
هسا             | الآن            | الآ       
لازم            | لازم            | لزم       
نشوف       

In [14]:
import re
import os
from nltk.stem.isri import ISRIStemmer


In [15]:
dictionary_content = """إحنا:نحن
بدنا:نريد
عشان:لأجل
الزلمة:الرجل
مبارح:أمس
مرة:جدا
إيش:ماذا
نسوي:نفعل
هسا:الآن
نشوف:نرى
ياخي:يا أخي
شلون:كيف"""

with open("dialects.txt", "w", encoding="utf-8") as f:
    f.write(dictionary_content)


In [36]:

# --- بداية الكلاس المطور ---

class SmartISRI_Pro(ISRIStemmer):
    def __init__(self, dict_path):
        super().__init__()
        self.dialect_map = {} # القاموس يبدأ فارغاً
        self.load_dictionary(dict_path) # نطلب منه تعبئة نفسه فوراً

    def load_dictionary(self, path):
        """دالة لقراءة الملف الخارجي وتعبئة القاموس"""
        print(f" جارِ تحميل القاموس من: {path} ...")
        
        if not os.path.exists(path):
            print("تحذير: الملف غير موجود! سأعمل بدون قاموس لهجات.")
            return

        try:
            with open(path, 'r', encoding='utf-8') as f:
                for line in f:
                    # تنظيف السطر من المسافات الزائدة
                    line = line.strip()
                    if not line or ":" not in line:
                        continue # تخطي الأسطر الفارغة أو الخطأ
                    
                    # تقسيم السطر بناءً على النقطتين
                    key, value = line.split(":")
                    self.dialect_map[key] = value
            
            print(f" تم تحميل {len(self.dialect_map)} كلمة بنجاح.")
            
        except Exception as e:
            print(f"حدث خطأ أثناء قراءة الملف: {e}")

    def smart_stem(self, word):
        # 1. تنظيف
        word_clean = re.sub("[إأآ]", "ا", word)
        
        # 2. بحث في القاموس الذي تم تحميله
        msa_word = self.dialect_map.get(word_clean, word_clean)
        
        # 3. تجذير
        root = super().stem(msa_word)
        return msa_word, root


In [37]:

# لاحظ: نمرر اسم الملف عند إنشاء الكائن
processor = SmartISRI_Pro("dialects.txt")

# نص جديد للتجربة
text = "ياخي شلون بدنا نخلص المشروع ؟"

print("\n" + "="*50)
for word in text.split():
    msa, root = processor.smart_stem(word)
    print(f"الكلمة: {word:<10} -> الفصحى: {msa:<10} -> الجذر: {root}") 



 جارِ تحميل القاموس من: dialects.txt ...
 تم تحميل 12 كلمة بنجاح.

الكلمة: ياخي       -> الفصحى: يا أخي     -> الجذر:  أخ
الكلمة: شلون       -> الفصحى: كيف        -> الجذر: كيف
الكلمة: بدنا       -> الفصحى: نريد       -> الجذر: نرد
الكلمة: نخلص       -> الفصحى: نخلص       -> الجذر: خلص
الكلمة: المشروع    -> الفصحى: المشروع    -> الجذر: شرع
الكلمة: ؟          -> الفصحى: ؟          -> الجذر: ؟
